In [2]:
import pandas as pd
from datetime import timedelta
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, r2_score
import joblib
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [3]:
# Example for tab-separated file
df = pd.read_csv('stop_times.txt')

# Save as CSV
df.to_csv('stop_times.csv')

# Drop multiple columns
df = df.drop(['continuous_pickup','continuous_drop_off','stop_headsign','pickup_type','drop_off_type'], axis=1)

In [4]:
# Convert time columns
# Convert times as duration from midnight
df['departure_time'] = pd.to_timedelta(df['departure_time'])
df['arrival_time'] = pd.to_timedelta(df['arrival_time'])

In [5]:

# Calculate travel time (in seconds)
df['travel_time_sec'] = (df['departure_time'] - df['arrival_time']).dt.total_seconds().abs()
# Calculate distance difference to previous stop
df['distance_m'] = df.groupby('trip_id')['shape_dist_traveled'].diff().fillna(0)

In [7]:
# Calculate average speed
df['speed_kmph'] = (df['distance_m'] / 1000) / (df['travel_time_sec'] / 3600 + 1e-5)  # avoid div by zero

In [8]:
# Extract time features
df['hour'] = df['departure_time'].dt.components['hours']
df['minute'] = df['departure_time'].dt.components['minutes']

In [9]:
# Features and label
features = df[['hour', 'minute', 'stop_id', 'distance_m']]
label = df['travel_time_sec']

In [10]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.2, random_state=42)

In [11]:
# Train Random Forest model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [13]:
# Evaluate
predictions = model.predict(X_test)
mae = mean_absolute_error(y_test, predictions)

(mae, df[['hour', 'minute', 'stop_id', 'distance_m', 'travel_time_sec']].head())

(0.0,
    hour  minute  stop_id  distance_m  travel_time_sec
 0     5      28       21       0.000             20.0
 1     5      31       20    1202.405             20.0
 2     5      33       19    1278.345             20.0
 3     5      35       18     834.186             20.0
 4     5      38       17     985.280             20.0)

In [14]:
df.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,shape_dist_traveled,timepoint,travel_time_sec,distance_m,speed_kmph,hour,minute
0,0,0 days 05:28:08,0 days 05:28:28,21,0,0.000,1,20.0,0.000,0.000000,5,28
1,0,0 days 05:30:58,0 days 05:31:18,20,1,1202.405,1,20.0,1202.405,216.044021,5,31
2,0,0 days 05:33:28,0 days 05:33:48,19,2,2480.750,1,20.0,1278.345,229.688660,5,33
3,0,0 days 05:35:33,0 days 05:35:53,18,3,3314.936,1,20.0,834.186,149.883689,5,35
4,0,0 days 05:37:53,0 days 05:38:13,17,4,4300.216,1,20.0,985.280,177.031743,5,38


<h1> Merging the stop data set for lat/lon </h1>

In [19]:
stops = pd.read_csv("stops.txt")

In [20]:
stops.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon
0,1,NaN,Dilshad Garden,NaN,28.675991,77.321495
1,2,NaN,Jhilmil,NaN,28.675648,77.312393
2,3,NaN,Mansrover park,NaN,28.675352,77.301178
3,4,NaN,Shahdara,NaN,28.673531,77.287270
4,5,NaN,Welcome,NaN,28.671986,77.277931


In [21]:
# Merge lat/lon into stop_times
merged = pd.merge(df, stops[['stop_id', 'stop_lat', 'stop_lon']], on='stop_id', how='left')

In [22]:
merged = merged.sort_values(by=['trip_id', 'stop_sequence']).reset_index(drop=True)
merged.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,shape_dist_traveled,timepoint,travel_time_sec,distance_m,speed_kmph,hour,minute,stop_lat,stop_lon
0,0,0 days 05:28:08,0 days 05:28:28,21,0,0.000,1,20.0,0.000,0.000000,5,28,28.720821,77.105042
1,0,0 days 05:30:58,0 days 05:31:18,20,1,1202.405,1,20.0,1202.405,216.044021,5,31,28.715008,77.115746
2,0,0 days 05:33:28,0 days 05:33:48,19,2,2480.750,1,20.0,1278.345,229.688660,5,33,28.707941,77.125732
3,0,0 days 05:35:33,0 days 05:35:53,18,3,3314.936,1,20.0,834.186,149.883689,5,35,28.703180,77.132355
4,0,0 days 05:37:53,0 days 05:38:13,17,4,4300.216,1,20.0,985.280,177.031743,5,38,28.697943,77.140465


<h>Calculate time taken to travel from one stop to the next </h1>

In [24]:
def time_to_sec(t):
    h, m, s = map(int, t.split(':'))
    return h * 3600 + m * 60 + s

import pandas as pd

# Ensure 'arrival_time' is timedelta
merged['arrival_time'] = pd.to_timedelta(merged['arrival_time'])

# Convert to seconds
merged['arrival_sec'] = merged['arrival_time'].dt.total_seconds()

# Calculate travel time to next stop
merged['next_arrival_sec'] = merged.groupby('trip_id')['arrival_sec'].shift(-1)
merged['travel_time_sec'] = merged['next_arrival_sec'] - merged['arrival_sec']


In [25]:
merged.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,shape_dist_traveled,timepoint,travel_time_sec,distance_m,speed_kmph,hour,minute,stop_lat,stop_lon,arrival_sec,next_arrival_sec
0,0,0 days 05:28:08,0 days 05:28:28,21,0,0.000,1,170.0,0.000,0.000000,5,28,28.720821,77.105042,19688.0,19858.0
1,0,0 days 05:30:58,0 days 05:31:18,20,1,1202.405,1,150.0,1202.405,216.044021,5,31,28.715008,77.115746,19858.0,20008.0
2,0,0 days 05:33:28,0 days 05:33:48,19,2,2480.750,1,125.0,1278.345,229.688660,5,33,28.707941,77.125732,20008.0,20133.0
3,0,0 days 05:35:33,0 days 05:35:53,18,3,3314.936,1,140.0,834.186,149.883689,5,35,28.703180,77.132355,20133.0,20273.0
4,0,0 days 05:37:53,0 days 05:38:13,17,4,4300.216,1,150.0,985.280,177.031743,5,38,28.697943,77.140465,20273.0,20423.0


In [26]:
merged['hour_of_day'] = merged['arrival_sec'] // 3600  # e.g., 5 AM = 5

<h1> Merging the trips  </h1>

In [28]:
trips = pd.read_csv("trips.txt")
merged = pd.merge(merged, trips[['trip_id', 'service_id', 'direction_id']], on='trip_id', how='left')

<h1> Clean Missing and Anomalous Values </h1>

In [33]:
# Remove rows where travel time is negative or null
cleaned = merged[(merged['travel_time_sec'] > 0) & (merged['travel_time_sec'].notnull())]

# Optional: Remove unreasonable travel times (> 30 minutes between stops)
cleaned = cleaned[cleaned['travel_time_sec'] < 1800]


In [34]:
cleaned.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,shape_dist_traveled,timepoint,travel_time_sec,distance_m,speed_kmph,hour,minute,stop_lat,stop_lon,arrival_sec,next_arrival_sec,hour_of_day,service_id,direction_id
0,0,0 days 05:28:08,0 days 05:28:28,21,0,0.000,1,170.0,0.000,0.000000,5,28,28.720821,77.105042,19688.0,19858.0,5.0,weekday,NaN
1,0,0 days 05:30:58,0 days 05:31:18,20,1,1202.405,1,150.0,1202.405,216.044021,5,31,28.715008,77.115746,19858.0,20008.0,5.0,weekday,NaN
2,0,0 days 05:33:28,0 days 05:33:48,19,2,2480.750,1,125.0,1278.345,229.688660,5,33,28.707941,77.125732,20008.0,20133.0,5.0,weekday,NaN
3,0,0 days 05:35:33,0 days 05:35:53,18,3,3314.936,1,140.0,834.186,149.883689,5,35,28.703180,77.132355,20133.0,20273.0,5.0,weekday,NaN
4,0,0 days 05:37:53,0 days 05:38:13,17,4,4300.216,1,150.0,985.280,177.031743,5,38,28.697943,77.140465,20273.0,20423.0,5.0,weekday,NaN


In [35]:
cleaned.to_parquet("gtfs_cleaned.parquet")

In [66]:
# Features
X = cleaned[[
    'hour_of_day',
    'stop_sequence',  # dst_sequence - src_sequence
    'distance_m',         # optional: haversine distance
    'trip_id',            # categorical
    'direction_id'         # 0 or 1
]]

# Target
y = cleaned['travel_time_sec']

In [70]:
# Preprocessing
categorical = ['trip_id']
numeric = ['hour_of_day', 'stop_sequence_diff', 'distance_m', 'direction_id']

preprocessor = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical)
], remainder='passthrough')

# Model pipeline
pipe = Pipeline([
    ('prep', preprocessor),
    ('model', LinearRegression())
])

pipe.fit(X, y)

# Save the model
import joblib
joblib.dump(pipe, 'models/lm_travel_time.pkl')

ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values